In [1]:
# Import the required libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import os
import joblib

In [2]:
total_approved_requests_per_city_and_year_df = pd.read_csv(os.path.abspath('../../') + '/data/total_requests_per_city_and_year_v3.csv')
total_approved_requests_per_city_and_year_df

,Yeshuv,SemelIishuv,year,total_requests,total_approved_requests,total_open_requests
0,אור יהודה,2400,2005,0.0,0.0,0
1,אור יהודה,2400,2006,0.0,0.0,0
2,אור יהודה,2400,2007,0.0,0.0,0
3,אור יהודה,2400,2008,1.0,0.0,0
4,אור יהודה,2400,2009,0.0,0.0,1
...,...,...,...,...,...,...
1075,תל אביב יפו,5000,2018,164.0,83.0,687
1076,תל אביב יפו,5000,2019,116.0,149.0,768
1077,תל אביב יפו,5000,2020,238.0,115.0,735
1078,תל אביב יפו,5000,2021,240.0,124.0,858


In [3]:
# Load your dataset and split it into training and testing sets
total_approved_requests_per_city_and_year_df = pd.concat([total_approved_requests_per_city_and_year_df] * 10, ignore_index=True)

total_approved_requests_per_city_and_year_df['SemelIishuv'] = total_approved_requests_per_city_and_year_df['SemelIishuv'].astype("category")
X_train, X_test, y_train, y_test = train_test_split(total_approved_requests_per_city_and_year_df[['SemelIishuv', 'year', 'total_open_requests']], 
                                                    total_approved_requests_per_city_and_year_df['total_approved_requests'],
                                                    test_size=0.2, random_state=42)

# Instantiate the Random Forest Regressor with desired parameters
rf_regressor = RandomForestRegressor(n_estimators=10000, random_state=42)

# Fit the model to the training data
rf_regressor.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf_regressor.predict(X_test)

# Evaluate the model using mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.00012717822222222145


### Save the model to a file

In [4]:
import joblib
joblib.dump(rf_regressor, 'random_forest_model_v2.pkl')

['random_forest_model_v2.pkl']

In [5]:
rf_regressor = joblib.load('random_forest_model_v2.pkl')

In [6]:
total_approved_requests_per_city_and_year_df['pred_total_approved_requests'] = rf_regressor.predict(
    total_approved_requests_per_city_and_year_df[['SemelIishuv', 'year', 'total_open_requests']])
total_approved_requests_per_city_and_year_df = total_approved_requests_per_city_and_year_df[
    ['SemelIishuv', 'Yeshuv', 'year', 'total_requests', 'total_open_requests', 'total_approved_requests', 'pred_total_approved_requests']]
total_approved_requests_per_city_and_year_df

,SemelIishuv,Yeshuv,year,total_requests,total_open_requests,total_approved_requests,pred_total_approved_requests
0,2400,אור יהודה,2005,0.0,0,0.0,0.0000
1,2400,אור יהודה,2006,0.0,0,0.0,0.0000
2,2400,אור יהודה,2007,0.0,0,0.0,0.0000
3,2400,אור יהודה,2008,1.0,0,0.0,0.0000
4,2400,אור יהודה,2009,0.0,1,0.0,0.0002
...,...,...,...,...,...,...,...
10795,5000,תל אביב יפו,2018,164.0,687,83.0,83.0000
10796,5000,תל אביב יפו,2019,116.0,768,149.0,148.9668
10797,5000,תל אביב יפו,2020,238.0,735,115.0,115.0238
10798,5000,תל אביב יפו,2021,240.0,858,124.0,124.0159


### plot

In [7]:
total_approved_requests_per_city_and_year_display_df = total_approved_requests_per_city_and_year_df.drop_duplicates()
import seaborn as sns
from matplotlib import pyplot as plt

sns.set_style("whitegrid")

from bidi.algorithm import get_display
total_approved_requests_per_city_and_year_display_df['Yeshuv'] = total_approved_requests_per_city_and_year_display_df['Yeshuv'].apply(get_display)
total_approved_requests_per_city_and_year_display_df['year'] = total_approved_requests_per_city_and_year_display_df['year'] - 2000
total_approved_requests_per_city_and_year_display_df['year'] = total_approved_requests_per_city_and_year_display_df['year'].astype(int)
sns.set(font_scale=3) 



total_approved_requests_per_city_and_year_display_df['pred_total_approved_requests'] = total_approved_requests_per_city_and_year_df['pred_total_approved_requests']

g = sns.FacetGrid(total_approved_requests_per_city_and_year_display_df, col="Yeshuv", col_wrap=3, height=10, ylim=(0, 150))
g.map(plt.plot, "year", "pred_total_approved_requests", linestyle="-", marker="o", color="blue")
g.map(plt.plot, "year", "total_approved_requests", linestyle="--", marker="x", color="red")
g.set_titles('{col_name}')
g.set_xlabels("years")
g.set_ylabels("total approved requests")
g.set(xticks=[5, 10, 15, 20], yticks=list(range(0, 151))[::30])

/var/folders/9d/pylsgxd55jj55h_1svvmnmjm0000gn/T/ipykernel_42105/1641836506.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_approved_requests_per_city_and_year_display_df['Yeshuv'] = total_approved_requests_per_city_and_year_display_df['Yeshuv'].apply(get_display)
/var/folders/9d/pylsgxd55jj55h_1svvmnmjm0000gn/T/ipykernel_42105/1641836506.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_approved_requests_per_city_and_year_display_df['year'] = total_approved_requests_per_city_and_year_disp

In [8]:
file_path = '/Users/sararozin/Documents/Tama/Tama/total_request_per_year/models/sir/original_sir_v2.ipynb'
%run $file_path

28 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 0, 2009: 0, 2010: 0, 2011: 0, 2012: 0, 2013: 1, 2014: 1, 2015: 0, 2016: 3, 2017: 2, 2018: 2, 2019: 1, 2020: 0, 2021: 0, 2022: 0}
save_sir_params 28 1.174697226916014 0.6324856833342344
70 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 1, 2007: 1, 2008: 2, 2009: 4, 2010: 7, 2011: 11, 2012: 15, 2013: 24, 2014: 23, 2015: 58, 2016: 41, 2017: 44, 2018: 43, 2019: 44, 2020: 20, 2021: 12, 2022: 21}
save_sir_params 70 1.1348391401591982 0.5864903724072146
166 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 0, 2009: 0, 2010: 0, 2011: 0, 2012: 0, 2013: 0, 2014: 0, 2015: 0, 2016: 0, 2017: 0, 2018: 0, 2019: 1, 2020: 0, 2021: 0, 2022: 0}
save_sir_params 166 0.81913045684113 0.3353959508063722
229 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 0, 2009: 0, 2010: 0, 2011: 0, 2012: 0, 2013: 1, 2014: 2, 2015: 0, 2016: 2, 2017: 1, 2018: 1, 2019: 0, 

6400 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 1, 2007: 2, 2008: 1, 2009: 3, 2010: 17, 2011: 11, 2012: 13, 2013: 44, 2014: 69, 2015: 22, 2016: 31, 2017: 41, 2018: 34, 2019: 13, 2020: 10, 2021: 4, 2022: 0}
save_sir_params 6400 1.2800179458413934 0.6605461983567428
6500 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 1, 2009: 1, 2010: 0, 2011: 0, 2012: 2, 2013: 9, 2014: 16, 2015: 20, 2016: 5, 2017: 3, 2018: 2, 2019: 2, 2020: 2, 2021: 0, 2022: 1}
save_sir_params 6500 1.3427597343933544 0.7227236455822259
6600 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 1, 2009: 1, 2010: 2, 2011: 11, 2012: 5, 2013: 31, 2014: 38, 2015: 73, 2016: 54, 2017: 36, 2018: 12, 2019: 27, 2020: 11, 2021: 2, 2022: 11}
save_sir_params 6600 1.225549095167978 0.6335240522752543
6700 {2000: 0, 2001: 0, 2002: 0, 2003: 0, 2004: 0, 2005: 0, 2006: 0, 2007: 0, 2008: 0, 2009: 0, 2010: 0, 2011: 0, 2012: 16, 2013: 3, 2014: 1, 2015: 0, 2016: 1, 20

<Figure size 640x480 with 0 Axes>

In [9]:
file_path = '/Users/sararozin/Documents/Tama/Tama/total_approved_request_per_year/preprocessing/feature_engineering_total_open_requests_v2.ipynb'
%run $file_path

In [10]:
from datetime import datetime


def guess_total_open_requests_futured_year(city, futured_year):
    current_year = datetime.now().year
    print('current_year', current_year)
    rf_regressor = joblib.load('random_forest_model_v2.pkl')
    tama_df = pd.read_csv('/Users/sararozin/Documents/Tama/Tama' + '/data/geocode_tama_address_data.csv')
    total_open_requests = calculate_total_open_request(city, current_year, tama_df)
    if futured_year > current_year:
        for year in range(current_year, futured_year):
            print('--------------', year, '-----------------')
            print('total_open_requests: ', total_open_requests)
            predict_total_approved_requests = rf_regressor.predict([[city, year, total_open_requests]])[0]
            print('predict_total_approved_requests:', predict_total_approved_requests)
            predict_total_new_requests = predict_total_requests_v2(city, year - 2000)
            print('predict_total_new_requests:', predict_total_new_requests)
            total_open_requests = total_open_requests - predict_total_approved_requests + predict_total_new_requests
    return total_open_requests

In [11]:

def predict_total_approved_requests(city, futured_year):
    guess_total_open_requests = guess_total_open_requests_futured_year(city, futured_year)
    total_approved_requests = rf_regressor.predict([[city, futured_year, guess_total_open_requests]])[0]
    if total_approved_requests > guess_total_open_requests:
        return guess_total_open_requests
    return total_approved_requests


### example code

In [12]:
predict_total_approved_requests(5000, 2027)

current_year 2023
-------------- 2023 -----------------
total_open_requests:  0


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


predict_total_approved_requests: 0.8612
predict_total_new_requests: 109
-------------- 2024 -----------------
total_open_requests:  108.1388


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


predict_total_approved_requests: 14.5947
predict_total_new_requests: 113
-------------- 2025 -----------------
total_open_requests:  206.54410000000001


...

predict_total_approved_requests: 43.0796
predict_total_new_requests: 117
-------------- 2026 -----------------
total_open_requests:  280.46450000000004


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


predict_total_approved_requests: 41.3324
predict_total_new_requests: 122


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


31.8284

In [13]:
file_path = '/Users/sararozin/Documents/Tama/Tama/total_approved_request_per_year/models/random_forest/zoom_out.ipynb'
%run $file_path

zoom out בני ברק
nearest city רמת גן


In [14]:
def predict_total_approved_requests_v2(city, futured_year):
    city = get_city_code(zoom_out(city))
    print('nearest city code:', city)
    return predict_total_approved_requests(city, futured_year)

In [15]:
city = 'אחיסמך'
predict_total_approved_requests_v2(city, 2026)

zoom out אחיסמך
nearest city לוד
nearest city code: 7000
current_year 2023
-------------- 2023 -----------------
total_open_requests:  0


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


predict_total_approved_requests: 0.0
predict_total_new_requests: 0
-------------- 2024 -----------------
total_open_requests:  0.0


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


predict_total_approved_requests: 0.0
predict_total_new_requests: 0
-------------- 2025 -----------------
total_open_requests:  0.0


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


predict_total_approved_requests: 0.0
predict_total_new_requests: 0


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


0.0